## Genome Annotation Viewer

This notebook contains code for viewing genome annotations in the form of gff or genbank files.

Links 

* https://bokeh.pydata.org/en/latest/docs/user_guide.html
* https://realpython.com/python-data-visualization-bokeh/

In [1]:
import os, sys, io, random
import string
import numpy as np
import pandas as pd

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

from IPython.display import HTML

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, CustomJS, Slider, HoverTool, NumeralTickFormatter, Arrow, NormalHead
from bokeh.models import LinearAxis, Range1d, DataRange1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot, column
import panel as pn
import panel.widgets as pnw
pn.extension()

from pybioviz import dashboards, utils, plotters
from importlib import reload

## plot features

In [2]:
def random_colors(size, seed=8):
    """random list of html colors of length sizes"""
    random.seed = seed
    from bokeh.palettes import Spectral8
    idx = np.random.randint(0,8, size)
    colors = [Spectral8[i] for i in idx]
    #colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    #            for i in range(size)]
    return colors

random_colors(10)

['#abdda4',
 '#3288bd',
 '#f46d43',
 '#f46d43',
 '#3288bd',
 '#3288bd',
 '#d53e4f',
 '#fee08b',
 '#fee08b',
 '#f46d43']

In [3]:
reload(utils)

def plot_features(features, start=0, end=None, fontsize="8pt", plot_width=800, plot_height=150,
                  tools="xpan, xwheel_zoom, save", color='#abdda4', rows=3, key='gene'):
    """Bokeh sequence alignment view"""
    
    df = utils.features_to_dataframe(features)#, cds=True)    
    df = df[(df.type!='region') & (df['type']!='source')]   
    df['length'] = df.end-df.start
    #df['level'] = 1
    #df['color'] = random_colors(len(df)) #'green'
    df['x'] = df.start+df.length/2
    df = df.fillna('')
    def get_arrow(x):
        if x.strand == 1:
            return x.end
        else:
            return x.start

    df['arrow_start'] = df.apply(get_arrow,1)
    df['arrow_end'] = df.apply(lambda x: x.arrow_start+50 if x.strand==1 else x.arrow_start-50, 1)
    
    #def get_y(x):
    #    df['col'].shift()
    np.random.seed(8) 
    
    #df['y'] = np.random.randint(1,9, len(df))
    y = list(range(0,rows)) * len(df)
    df['y'] = y[:len(df)]
    
    if end == None:
        end = df.end.max()+100
        if end>10000:
            end=10000
    #print (df[:3])
    text = df.gene
    S = df.start.min()
    N = df.end.max()+10        
    x = list(df.start+df.length/2)
    h = 20

    source = ColumnDataSource(df)
    x_range = Range1d(start,end,min_interval=1)  
    viewlen = end-start
    hover = HoverTool(
        tooltips=[            
            ("gene", "@gene"),     
            ("locus_tag", "@locus_tag"),
            ("product", "@product"), 
            ("strand", "@strand"),
            ("length", "@length"),             
        ],        
    )  
    tools=[hover, tools]    
    #sequence text view with ability to scroll along x axis
    p = figure(title=None, plot_width=plot_width, plot_height=plot_height, x_range=x_range,
                y_range=(-1,rows), tools=tools, min_border=0, toolbar_location='right')#, lod_factor=1)
    #display text only at certain zoom level?
    #print (viewlen)
    if viewlen<20000:
        tags = Text(x="x", y="y", y_offset=-8, text=key, text_align='center',text_color="black", 
                     text_font="monospace",text_font_size=fontsize, name="genetext")
        p.add_glyph(source, tags)
    #rects
    rects = Rect(x="x", y="y", width="length", height=.4, fill_color=color, fill_alpha=0.4, name='rects')
    #arrows
    arr = Arrow(source=source, x_start="arrow_start", x_end="arrow_end", y_start="y", y_end="y", 
                line_color="black", name='arrows', end=NormalHead(size=8))
    p.add_glyph(source, rects)    
    p.add_layout(arr)
    
    p.grid.visible = False
    p.yaxis.visible = False
    p.xaxis.major_label_text_font_style = "bold"
    p.yaxis.minor_tick_line_width = 0
    p.yaxis.major_tick_line_width = 0
    p.toolbar.logo = None
    p.xaxis.formatter = NumeralTickFormatter(format="(0,0)")
    return p


feats = utils.gff_to_features('RD900MAF.gff')
#feats = utils.genbank_to_features('1765.416.gbk',key=12)
#feats = utils.gff_to_features('Mbovis_AF212297.gff')
p = plot_features(features=feats, start=100)
pn.pane.Bokeh(p)

TypeError: 'NoneType' object is not iterable

In [ ]:
reload(utils)
df = utils.features_to_dataframe(feats)#, cds=True)
df[:4]
#feats[3]

## viewer app

In [ ]:
reload(plotters)

def genome_features_viewer(gff_file, ref_file=None, plot_width=900):
    """Gene feature viewer app"""
    
    if gff_file is None:
        return
    
    features = utils.gff_to_features(gff_file)
    df = utils.features_to_dataframe(features)
    
    loc_pane = pnw.TextInput(name='location',value='',width=150)
    search_pane = pnw.TextInput(name='find_gene',value='',width=220)
    slider = pnw.IntSlider(name='start',start=0,end=10000,step=500,value=1,width=plot_width)
    xzoom_slider = pnw.IntSlider(name='zoom',start=1,end=500,value=100,step=5,width=100)
    chrom_select = pnw.Select(name='chrom',width=220)
    left_button = pnw.Button(name='<',width=40)
    right_button = pnw.Button(name='>',width=40)
    
    feature_pane = pn.pane.Bokeh(height=100,margin=10)
    seq_pane = pn.pane.Bokeh(height=50,margin=10)
    debug_pane = pn.pane.Str('debug',width=200,style={'background':'yellow','margin': '4pt'})
    
    if ref_file is not None:
        seqlen = utils.get_fasta_length(ref_file)
        slider.end = seqlen
    else:
        slider.end = int(df.end.max())

    def search_features(event):
        """Find a feature"""
        
        term = search_pane.value        
        feats = utils.gff_to_features(gff_file)
        df = utils.features_to_dataframe(feats)    
        df['gene'] = df.gene.fillna('')
        f = df[df.gene.str.contains(term)].iloc[0]
        #debug_pane.object = str(f.start)
        slider.value = int(f.start)-100
        update(event)
        return
    
    def pan(event):
        p = feature_pane.object
        rng = p.x_range.end-p.x_range.start        
        inc = int(rng/10)
        print (event.obj.name)
        if event.obj.name == '<':
            slider.value = int(slider.value) - inc        
        else:
            slider.value = int(slider.value) + inc   
        update(event)
        return
    
    def update(event):      
        print (event.obj.name)
        if event.obj.name in ['start', 'zoom']:
            xzoom = xzoom_slider.value*200
            start = int(slider.value)
            N = xzoom/2
            end = int(start+N)
            loc_pane.value = str(start)+':'+str(end)            
        elif event.obj.name == 'location':            
            vals = loc_pane.value.split(':')
            start = int(vals[0])
            end = int(vals[1])
            slider.value = start        
            
        #debug_pane.object=type(start)
        p = feature_pane.object
        p.x_range.start = start
        p.x_range.end = end
        if ref_file:
            sequence = utils.get_fasta_sequence(ref_file, start, end)
            seq_pane.object = plotters.plot_sequence(sequence, plot_width, plot_height=50,fontsize='9pt',xaxis=False)            
        return
        
    slider.param.watch(update,'value',onlychanged=True)
    #slider.param.trigger('value')    
    xzoom_slider.param.watch(update,'value')       
    search_pane.param.watch(search_features,'value')    
    loc_pane.param.watch(update,'value',onlychanged=True)    
    left_button.param.watch(pan,'clicks')
    right_button.param.watch(pan,'clicks')
    #debug_pane.object = utils.get_fasta_names(ref_file)[0]
    if ref_file != None:
        chrom_select.options = utils.get_fasta_names(ref_file)
    #plot
    p = feature_pane.object = plot_features(features, 0, 10000, plot_width=plot_width, tools="", rows=4)
    
    #side = pn.Column(file_input,css_classes=['form'],width=200,margin=20)
    top = pn.Row(loc_pane,xzoom_slider,search_pane,chrom_select,left_button,right_button)
    main = pn.Column(feature_pane, seq_pane, sizing_mode='stretch_width')
    app = pn.Column(top,slider,main,debug_pane, sizing_mode='stretch_width',width_policy='max',margin=20)
    return app

app = genome_features_viewer('Mbovis_AF212297.gff')#, 'Mbovis-AF212297.fa')
app

In [ ]:
def gff_to_features(gff_file, target_lines=None, limit_info=None):
    """Get features from gff file"""

    if gff_file is None or not os.path.exists(gff_file):
        return
    from BCBio import GFF
    in_handle = open(gff_file,'r')
    rec = list(GFF.parse(in_handle, target_lines=target_lines))[0]
    in_handle.close()
    return rec.features

limit_info = dict(
        gff_id = ["1"],
        gff_source = ["Coding_transcript"])
gff_file='/storage/genomes/cow/UMD_3.1.1_renamed.gff3'
feats = gff_to_features(gff_file,target_lines=10,limit_info=limit_info)
print (len(feats))

## tabix for random access

In [ ]:
import tabix
gff_file='/storage/genomes/cow/UMD_3.1.1_renamed.sorted.gff3'
tb=tabix.open(gff_file)


In [ ]:
records = tb.query("1", 1000000, 1250000)

In [ ]:
def tabix_index(filename,
        preset="gff", chrom=1, start=4, end=5, skip=0, comment="#"):
    """Call tabix to create an index for a bgzip-compressed file."""
    from subprocess import Popen
    Popen(['tabix', '-p', preset, '-s', chrom, '-b', start, '-e', end,
        '-S', skip, '-c', comment])
    
tabix_index(gff_file)

In [ ]:
Range1d(0,3,min_interval=1)